In [2]:
import numpy as np
import chainer
from chainer import Variable, cuda, Function, report, training, utils
from chainer import Link, Chain, ChainList, datasets
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

import random
import matplotlib.pyplot as plt
import matplotlib.cm as cm

train, test = datasets.get_mnist(ndim=3)

KeyboardInterrupt: 

In [4]:
class ConxNeuralNet(Chain):
    def __init__(self):
        super(ConxNeuralNet, self).__init__(
        cn1 = L.Convolution2D(1,20,5),
        cn2 = L.Convolution2D(20,50,5),
        l1 = L.Linear(800,500),
        l2 = L.Linear(500,10)
        )
    def __call__(self, x, t):
        return F.softmax_cross_entropy(self.fwd(x),t)
    def fwd(self,x):
        h1 = F.max_pooling_2d(F.relu(self.cn1(x)),2)
        h2 = F.max_pooling_2d(F.relu(self.cn2(h1)),2)
        h3 = F.dropout(F.relu(self.l1(h2)))
        return self.l2(h3)

In [5]:
CVmodel = ConxNeuralNet()

from chainer import iterators

optimizer = chainer.optimizers.Adam()
optimizer.setup(CVmodel)

iterator = iterators.SerialIterator(train, 1000)

updater = training.StandardUpdater(iterator, optimizer)
trainer = training.Trainer(updater, (10, 'epoch'))
trainer.extend(extensions.ProgressBar())
trainer.run()

     total [########..........................................] 16.67%
this epoch [#################################.................] 66.67%
       100 iter, 1 epoch / 10 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [################..................................] 33.33%
this epoch [################..................................] 33.33%
       200 iter, 3 epoch / 10 epochs
   0.51381 iters/sec. Estimated time to finish: 0:12:58.495275.
     total [#########################.........................] 50.00%
this epoch [..................................................]  0.00%
       300 iter, 5 epoch / 10 epochs
   0.51504 iters/sec. Estimated time to finish: 0:09:42.481162.
     total [#################################.................] 66.67%
this epoch [#################################.................] 66.67%
       400 iter, 6 epoch / 10 epochs
   0.51612 iters/sec. Estimated time to finish: 0:06:27.506013.
     total [#######################

In [6]:
ok = 0
for i in range(len(test)): 
    x = Variable(np.array([test[i][0]], dtype=np.float32))
    t = test[i][1]
    out = CVmodel.fwd(x)
    ans = np.argmax(out.data)
    if (ans == t):
        ok +=1
print((ok*1.0/len(test)))

0.9881
